In [1]:
import numpy as np


In [2]:
def resistorChain_matrix(N):
    ''' 
    Returns the associated matrix A for a triangular resistor chain lattice with N+2 nodes
    '''
    A= np.eye(N)
    if N==2:
        A[0], A[1] = [3,-1], [-1,3]
    if N==3:
        A = np.array([3,-1,-1], [-1,4,-1], [-1,-1,3])
    elif N >= 4:
        A[0, 0:3], A[1, 0:4]=[3,-1,-1], [-1,4,-1,-1]
        A[N-1, -3::], A[N-2, -4::] = [-1,-1,3], [-1,-1,4,-1]
        for i in range(2, N-2):
            A[i, i-2:i+3] = [-1,-1,4,-1,-1]
    return A
def resistorChain_w(N, v_in):
    ''' 
    Returns the associated vector w for a triangular resistor chain lattice with N+2 nodes
    '''
    w=np.zeros(N)
    if N==1:
        w[0]=0.5*v_in 
    else:
        w[0:2] = [v_in]*2 
    return w 

resistorChain_matrix(6)

array([[ 3., -1., -1.,  0.,  0.,  0.],
       [-1.,  4., -1., -1.,  0.,  0.],
       [-1., -1.,  4., -1., -1.,  0.],
       [ 0., -1., -1.,  4., -1., -1.],
       [ 0.,  0., -1., -1.,  4., -1.],
       [ 0.,  0.,  0., -1., -1.,  3.]])

In [3]:
class ResistorChain:
    def __init__(self, N, v_in):
        self.N, self.v_in = N, v_in 
    def get_N(self):
        return self.N 
    def get_Vin(self):
        return self.v_in 
    def get_A(self):
        N = self.N 
        A = np.eye(N)
        if N==2:
            A[0], A[1] = [3,-1], [-1,3]
        if N==3:
            A = np.array([3,-1,-1], [-1,4,-1], [-1,-1,3])
        elif N >= 4:
            A[0, 0:3], A[1, 0:4]=[3,-1,-1], [-1,4,-1,-1]
            A[N-1, -3::], A[N-2, -4::] = [-1,-1,3], [-1,-1,4,-1]
            for i in range(2, N-2):
                A[i, i-2:i+3] = [-1,-1,4,-1,-1]
        return A
    def get_w(self):
        N, v_in= self.N, self.v_in
        w = np.zeros(N)
        if N==1:
            w[0]=0.5*v_in 
        else:
            w[0:2] = [v_in]*2 
        return w 
            

            

In [4]:
def ref(test, b):
    test, b = np.array(test).astype(float), np.array(b).astype(float) #Converts to float
    b = np.reshape(b, (-1,1))
    N = np.shape(test)[0]
    augment = np.append(test, b, axis=-1)
    for c in range(N-1):
        if augment[c,c] == 0: #Check if pivot is zero.
            #Check for nonzero first entries in the lower rows.
                pointer = c+1 
                while pointer < N: #checks rows up to the last row of the matrix
                    if augment[pointer, c] != 0: #Swap with the pivot row the lower row if first entry is nonzero.
                        copy = np.copy(augment[c])
                        augment[c], augment[pointer] = augment[pointer], copy 
                        break 
                    else: #If not, move pointer to the next lower row
                        pointer = pointer + 1
        if augment[c,c] != 0: #Check if the pivot is 0. If not, then proceed to row reduce the lower rows.
            augment[c] = augment[c]/augment[c,c]
            for r in range(c+1, N):
                factor = augment[r,c]/augment[c,c]
                augment[r] = augment[r] - augment[c]*factor 
    if augment[N-1,N-1] != 0: #Divide the last row by the leading coefficient if it is nonzero. 
        augment[N-1] = augment[N-1]/(augment[N-1,N-1])
    return augment

def gauss_elim(A, b):
    N = np.shape(A)[0]
    assert N == np.shape(A)[1], 'Non-square matrix A, no unique solutions'
    aug = ref(A, b)
    assert np.sum(aug[N-1,0:-1]) != 0, 'Matrix linearly dependent, infinitely many solutions'
    for i in range(N-1, 0, -1):
        for m in range(i):
            factor = aug[m,i]/aug[i,i]
            aug[m] = aug[m] - factor*aug[i]
    return aug[:,N]



In [8]:
# Using homebrew gaussian elimination (slower)
N=1000
v_in =5
A = resistorChain_matrix(N)
v = resistorChain_w(N, v_in)
gauss_elim(A,v)

array([4.98885075, 4.98621877, 4.98033349, 4.97569085, 4.97057357,
       4.96563758, 4.96063235, 4.95565357, 4.95066468, 4.94567965,
       4.94069315, 4.93570721, 4.93072106, 4.92573498, 4.92074888,
       4.91576279, 4.91077669, 4.9057906 , 4.90080451, 4.89581841,
       4.89083232, 4.88584622, 4.88086013, 4.87587403, 4.87088794,
       4.86590184, 4.86091575, 4.85592966, 4.85094356, 4.84595747,
       4.84097137, 4.83598528, 4.83099918, 4.82601309, 4.82102699,
       4.8160409 , 4.8110548 , 4.80606871, 4.80108262, 4.79609652,
       4.79111043, 4.78612433, 4.78113824, 4.77615214, 4.77116605,
       4.76617995, 4.76119386, 4.75620777, 4.75122167, 4.74623558,
       4.74124948, 4.73626339, 4.73127729, 4.7262912 , 4.7213051 ,
       4.71631901, 4.71133291, 4.70634682, 4.70136073, 4.69637463,
       4.69138854, 4.68640244, 4.68141635, 4.67643025, 4.67144416,
       4.66645806, 4.66147197, 4.65648587, 4.65149978, 4.64651369,
       4.64152759, 4.6365415 , 4.6315554 , 4.62656931, 4.62158

In [16]:
# Using linalg module
from numpy.linalg import solve 
N, v_in = 1000, 5
A = resistorChain_matrix(N)
v = resistorChain_w(N, v_in)
x = solve(A, v)


In [10]:
np.arange(15)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [17]:
A = np.reshape(np.arange(15), (5,3))
v= np.reshape(np.arange(5), (5,1))


In [18]:
x=solve(A,v)

LinAlgError: Last 2 dimensions of the array must be square